In [10]:
import numpy as np
from scipy.stats import ttest_rel
import pandas as pd

In [30]:
def compute_frac_sig(feature_extraction_arr, dataset, figure_num):
    
    df = pd.read_csv(f'/home2/ebrahim/beyond-brainscore/analyze_results/figures_code/figures_data/figure{figure_num}/pvalues_{dataset}.csv')
    

    for fe in feature_extraction_arr: 
        
        df_fe = df.loc[df.fe==fe].copy()
        df_fe['pval_sig'] = np.where(df_fe['pval']<0.05, 1, 0)
        
        # Calculate the mean proportion of significant p-values per subject
        subject_means = df_fe.groupby('subject')['pval_sig'].mean()

        # Calculate the overall mean and standard error
        overall_mean = subject_means.mean() * 100  # Convert to percentage
        std_error = (subject_means.std() / np.sqrt(len(subject_means))) * 100  # Convert to percentage

        # Print the results with two significant digits
        print(f"{overall_mean:.3g} {std_error:.3g}%")

In [12]:
df = pd.read_csv('/home2/ebrahim/beyond-brainscore/analyze_results/figures_code/figures_data/figure5/pvalues_pereira.csv')
df

,Unnamed: 0,fe,subject,pval,pval_orig,network
0,0,-lt,18,8.347298e-54,3.700926e-54,language
1,1,-lt,18,1.634066e-51,1.055689e-51,language
2,2,-lt,18,3.911878e-61,1.690677e-62,language
3,3,-lt,18,1.452273e-45,1.309427e-45,language
4,4,-lt,18,1.606048e-42,1.493553e-42,language
...,...,...,...,...,...,...
40498,40498,-sp,426,4.309963e-48,4.059052e-48,language
40499,40499,-sp,426,1.078392e-55,4.847601e-56,language
40500,40500,-sp,426,1.559072e-55,7.307071e-56,language
40501,40501,-sp,426,3.165618e-52,2.419120e-52,language


In [13]:
compute_frac_sig(['-lt', '-mp', '-sp'], 'pereira',2)
compute_frac_sig(['-lt', '-mp', '-sp'], 'fedorenko',2)
compute_frac_sig(['-lt', '-mp', '-sp'], 'blank',2)

21 8.7%
17 7.3%
22 9.2%
29 9.6%
29 11%
23 9.8%
0 0%
1.7 1.7%
1.7 1.7%


In [32]:
compute_frac_sig(['-lt', '-mp', '-sp'], 'pereira', 4)
compute_frac_sig(['-lt', '-mp', '-sp'], 'fedorenko', 4)
compute_frac_sig(['-lt', '-mp', '-sp'], 'blank', 4)

0.0447 0.0447%
0.443 0.418%
0.679 0.646%
4.67 2.12%
3.33 2.22%
4.67 2.12%
1.67 1.67%
3.33 2.04%
1.67 1.67%


In [33]:
compute_frac_sig(['-lt', '-mp', '-sp'], 'pereira',5)
compute_frac_sig(['-lt', '-mp', '-sp'], 'fedorenko',5)
compute_frac_sig(['-lt', '-mp', '-sp'], 'blank',5)

0 0%
0 0%
0 0%
0 0%
0 0%
0 0%
0 0%
0 0%
0 0%


In [21]:
df = pd.read_csv('/home2/ebrahim/beyond-brainscore/analyze_results/figures_code/figures_data/figure5/pvalues_pereira.csv')
df['pval'][0]

8.347297570238016e-54

In [27]:
np.load('/data/LLMs/brainscore/results_pereira/shuffled/pereira_gpt2-xl_OASM_384_layer1_1000_384.npz')

NpzFile '/data/LLMs/brainscore/results_pereira/shuffled/pereira_gpt2-xl_OASM_384_layer1_1000_384.npz' with keys: pnum, out_of_sample_r2, pearson_r